# Assignment 13

## Neo4J

In [218]:
%%bash
docker run \
    -d --name neo4j \
    -v "$(pwd)/dbfiles:/var/lib/neo4j/import/dbfiles" \
    --rm \
    --publish=7474:7474 \
    --publish=7687:7687 \
    --env NEO4J_AUTH=neo4j/fancy!99Doorknob \
    neo4j

#  --env=NEO4J_dbms_memory_pagecache_size=1G \
#    --env=NEO4J_dbms_memory_heap_initial_size=1G \
#    --env=NEO4J_dbms_memory_heap_max_size=1G \

d9ca9fa013328dc08d3aff6a947b9dc26545a3367a796d6765f723e7a25146ea


In [220]:
import sys
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
auth=("neo4j", "fancy!99Doorknob")
driver = GraphDatabase.driver(uri, auth=auth)

def neo(command):
    try:
        with driver.session() as session:
            result = session.run(command)
        return result # result is a resultset/cursor for neo4j
    except Exception as ex:
        print(str(ex), file=sys.stderr)
        
def neov(command):
    try:
        return neo(command).values()
    except Exception as ex:
        return 'Shit happened'
"done"

'done'

## MySQL

In [206]:
%%bash
docker run \
    --rm \
    --name my_mysql \
    -v "$(pwd)/mysql_dbfiles:/var/lib/mysql" \
    -p 3307:3307 \
    -e MYSQL_ROOT_PASSWORD=deterentysker!42snapsnap \
    -d \
    mysql

b14b66e6071e2333ebb54ae8b3858893659285f844a86c92522ae9e20192fea8


In [213]:
import sys
import mysql.connector

def rootconnect():
    try:
        pw = 'deterentysker!42snapsnap'
        conn = mysql.connector.connect( host='localhost', database='copenhagen',user='root', password=pw)
        conn.autocommit = True
        return conn;
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    

conn = rootconnect()

def sqlQuery(sqlString):
    global conn
    try:
        if not conn.is_connected():
            conn = rootconnect()
        cursor = conn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchall()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

ModuleNotFoundError: No module named 'mysql'

In [223]:
neov('''
LOAD CSV WITH HEADERS FROM "file:///dbfiles/social_network_nodes_small.csv" AS row
MERGE (:User {id: row.node_id,
                  name: row.name,
                  job: row.job,
                  birthday: row.birthday})
CREATE INDEX ON :User(id)
''')

[]

In [224]:
neov('''
LOAD CSV WITH HEADERS FROM "file:///dbfiles/social_network_edges_small.csv" AS row
MATCH (source_user:User)
WHERE source_user.id = row.source_node_id
MATCH (target_user:User)
WHERE target_user.id = row.target_node_id
MERGE (source_user)-[endorsement:ENDORSES]->(target_user)
''')

[]

In [313]:
neov('''
MATCH (u:User)-[:ENDORSES]->(t:User)-[:ENDORSES]->(t2:User)-[:ENDORSES]->(t3:User)-[:ENDORSES]->(t4:User)-[:ENDORSES]->(t5:User)
WHERE t.id = "173"
RETURN count(*)
''')

[[5427]]

In [312]:
import random
import timeit


def select_twenty_random_nodes(seed):
    random.seed(seed)
    
    ids = [user_id[0] for user_id in neov('''
        MATCH (user:User)
        RETURN user.id
    ''')]
    
    return random.choices(ids, k=20)


def run_query(depth_level, user_id):
     neov(f'''
        MATCH (u:User)-[:ENDORSES]->(t1:User){depth_level}
        WHERE t1.id = '{user_id}'
        RETURN count(*)
     ''')

        
def run_endorsement_queries(seed):
    random_ids = select_twenty_random_nodes(seed)
    depth_counts = {}
    
    for depth in range(1, 6):
        for user_id in random_ids:
            test = range(int(depth) - 1)
            depth_level = "".join([f"-[:ENDORSES]->(t{idx + 2}:User)" for _, idx in enumerate(test)])
            
            time = timeit.Timer(lambda: run_query(depth_level, user_id))
            depth_counts.setdefault(depth, []).append(time.timeit(10))
    
    return depth_counts


print(run_endorsement_queries(10))

{1: [0.10562692099847482, 0.15403635099937674, 0.04964630200265674, 0.036589115999959176, 0.03224448100081645, 0.036095171999477316, 0.04207768099877285, 0.03419118799865828, 0.03324711800087243, 0.03556467699672794, 0.04525495299822069, 0.03589328099769773, 0.03901659400071367, 0.03158983899993473, 0.03229630400164751, 0.03302151099705952, 0.035677659998327726, 0.0317809269981808, 0.03167588599899318, 0.0413900619969354], 2: [0.03210382700126502, 0.053268482002749806, 0.0349928390023706, 0.028926199996931246, 0.035924274001445156, 0.03143623300275067, 0.030597558001318248, 0.03212376599913114, 0.029524656001740368, 0.055941022001206875, 0.04999104099988472, 0.03635705800115829, 0.038064332999056205, 0.12677659999826574, 0.1584704220003914, 0.04283816400129581, 0.03838394400008838, 0.025223689000995364, 0.02680109100037953, 0.03701365699816961], 3: [0.04226076099803322, 0.17370660999949905, 0.15307208800004446, 0.03169504900142783, 0.11278296800082899, 0.08345403100247495, 0.0859918860